<a href="https://colab.research.google.com/github/sulphatet/GetARoom_MLHackathlon/blob/main/roomRegress_clean.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
data =pd.read_csv("drive/My Drive/Data/train_DE1.csv")
test_data =pd.read_csv("drive/My Drive/Data/test_DE1.csv")

In [ ]:
from sklearn.model_selection import train_test_split
data_train,data_test = train_test_split(data)

In [ ]:
from sklearn import preprocessing
from sklearn.preprocessing import RobustScaler
from sklearn.experimental import enable_iterative_imputer  # noqa
# now you can import normally from sklearn.impute
from sklearn.impute import KNNImputer 
from sklearn.preprocessing import StandardScaler

data_train['Furnishing'] = data_train['Furnishing'].replace(['Semi_Furnished', 'Unfurnished', 'Fully Furnished'],[1,2,3])
data_train['Property_Type'] = data['Property_Type'].replace(['Apartment', 'Bungalow', 'Single-family home', 'Duplex','#R%$G&867', 'Container Home'],[1,2,3,4,0,5])
data_train['Water_Supply'] = data_train['Water_Supply'].replace(['Once in a day - Morning', 'Once in a day - Evening', 'All time','NOT MENTIONED', 'Once in two days'],[1,2,3,0,4])
data_train['Crime_Rate'] = data_train['Crime_Rate'].replace(['Slightly below average', 'Well below average','Well above average',  'Slightly above average'],[1,2,3,4])
data_train['Dust_and_Noise'] = data['Dust_and_Noise'].replace(['Medium',  'High', 'Low'],[2,3,1])
data_train['Power_Backup'] = data['Power_Backup'].replace(['No', 'Yes','NOT MENTIONED'],[0,1,-1])
#data_train['Frequency_of_Powercuts'] = data['Frequency_of_Powercuts'].replace([np.NaN]],[0.5])

numeric=['Furnishing', 'Property_Type', 'Water_Supply','Crime_Rate','Dust_and_Noise','Power_Backup']

#sc = StandardScaler()
#data_train[numeric]=sc.fit_transform(data_train[numeric])

#data_train = data_train.replace(np.NaN,0)

X_train = data_train.iloc[:,1:-1]
si = KNNImputer()
imp_mean = si.fit(X_train)
X_train = imp_mean.transform(X_train)



Y_train = data_train['Habitability_score']

In [ ]:
data_test['Furnishing'] = data['Furnishing'].replace(['Semi_Furnished', 'Unfurnished', 'Fully Furnished'],[1,2,3])
data_test['Property_Type'] = data['Property_Type'].replace(['Apartment', 'Bungalow', 'Single-family home', 'Duplex','#R%$G&867', 'Container Home'],[1,2,3,4,0,5])
data_test['Water_Supply'] = data['Water_Supply'].replace(['Once in a day - Morning', 'Once in a day - Evening', 'All time','NOT MENTIONED', 'Once in two days'],[1,2,3,0,4])
data_test['Crime_Rate'] = data['Crime_Rate'].replace(['Slightly below average', 'Well below average','Well above average',  'Slightly above average'],[1,2,3,4])
data_test['Dust_and_Noise'] = data['Dust_and_Noise'].replace(['Medium',  'High', 'Low'],[2,3,1])
data_test['Power_Backup'] = data['Power_Backup'].replace(['No', 'Yes','NOT MENTIONED'],[0,1,-1])
#data_train['Frequency_of_Powercuts'] = data['Frequency_of_Powercuts'].replace([np.NaN]],[0.5])

#data_test[numeric]=sc.fit_transform(data_test[numeric])
#data_test = data_test.replace(np.NaN,0)
X_test = data_test.iloc[:,1:-1]

si = KNNImputer()
imp_mean = si.fit(X_test)
X_test = imp_mean.transform(X_test)

Y_test = data_test['Habitability_score'] 

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
model = LinearRegression()
clf1 = model.fit(X_train,Y_train)
model = RandomForestRegressor()
clf2 = model.fit(X_train,Y_train)

In [ ]:
from sklearn.metrics import r2_score

In [ ]:
Y_pred = clf2.predict(X_test)
print(r2_score(Y_test,Y_pred))

0.8244727844777572


In [ ]:
#import seaborn as sns
#sns.pairplot(data=data_train, diag_kind='kde')

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor
import lightgbm as lgbm
import xgboost as xg
from sklearn.preprocessing import QuantileTransformer
from sklearn.compose import TransformedTargetRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error

#generic function to fit model and return metrics for every algorithm
def boost_models(x):
    #transforming target variable through quantile transformer
    regr_trans = TransformedTargetRegressor(regressor=x, transformer=None)
    regr_trans.fit(X_train, Y_train)
    yhat = regr_trans.predict(X_test)
    algoname= x.__class__.__name__
    return algoname, round(r2_score(Y_test, yhat),3), round(mean_absolute_error(Y_test, yhat),2), round(np.sqrt(mean_squared_error(Y_test, yhat)),2)

algo=[GradientBoostingRegressor(), lgbm.LGBMRegressor(), xg.XGBRFRegressor()]
score=[]
for a in algo:
    score.append(boost_models(a))

 #Collate all scores in a table
pd.DataFrame(score, columns=['Model', 'Score', 'MAE', 'RMSE'])